In [2]:
import sys
sys.path.append('../..')

import numpy as np
from galois_datasets import load_all_data_mnist, load_all_data_cifar10, load_all_data_fashion_mnist
from utils import create_batch_data, to_finite_field_domain, from_galois_to_real_domain
import modules
import galois_layers
from galois_criterions import GaloisFieldMSELoss
from sklearn.datasets import make_classification
import galois

In [3]:
BATCH_SIZE = 256
EPOCH = 1
PRINT = 1
FLATTEN = False
# 0, MNIST; 1, FashionMNIST; 2, CIFAR10; 3 RANDOM
DATASET_MODE = 0

QUANTIZATION_INPUT = 8
QUANTIZATION_WEIGHT = 16
QUANTIZATION_BATCH_SIZE = 8
LR = 7
PRIME = 684502462494449

In [4]:
field = galois.GF(PRIME)

In [5]:
# data fetching
load_path = '../../data'
if DATASET_MODE == 0:
    train_data, train_label, test_data, test_label = load_all_data_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, field, flatten=FLATTEN)
elif DATASET_MODE == 1:
    train_data, train_label, test_data, test_label = load_all_data_fashion_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, field, flatten=FLATTEN)
elif DATASET_MODE == 2:
    train_data, train_label, test_data, test_label = load_all_data_cifar10(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, field, flatten=FLATTEN)
elif DATASET_MODE == 3:
    train_data, train_label = make_classification(n_samples=10, n_features=100, n_classes=10, n_clusters_per_class=1, n_informative=10)
    train_data = train_data.reshape((-1, 4, 5, 5))
    test_data, test_label = train_data, train_label
    train_label = np.zeros((10, 10))
    for idx, label in enumerate(test_label):
        train_label[idx][label] = 1
    train_data, train_label, test_data = to_finite_field_domain(train_data, QUANTIZATION_INPUT, PRIME), to_finite_field_domain(train_label, QUANTIZATION_WEIGHT, PRIME), to_finite_field_domain(test_data, QUANTIZATION_INPUT, PRIME)
    train_data, train_label, test_data = field(train_data), field(train_label), field(test_data)
else:
    train_data, train_label, test_data, test_label = None, None, None, None
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

In [6]:
model_arr = [
    galois_layers.GaloisFieldPiNetSecondOrderConvLayer(4, 6, (3, 3), QUANTIZATION_WEIGHT, PRIME, field, first_layer=True, padding=(2, 2, 2, 2), quantization_bit_input=QUANTIZATION_INPUT),
    galois_layers.GaloisFieldPiNetSecondOrderConvLayer(6, 6, (5, 5), QUANTIZATION_WEIGHT, PRIME, field, padding=(2, 2, 2, 2)),
    galois_layers.GaloisFieldPiNetSecondOrderConvLayer(6, 6, (3, 3), QUANTIZATION_WEIGHT, PRIME, field),
    modules.Flatten(),
    galois_layers.GaloisFieldPiNetSecondOrderLinearLayer(150, 64, QUANTIZATION_WEIGHT, PRIME, field),
    galois_layers.GaloisFieldLinearLayer(64, 10, QUANTIZATION_WEIGHT, PRIME, field)
]

model = modules.Network(model_arr)
criterion = GaloisFieldMSELoss(PRIME, QUANTIZATION_WEIGHT, QUANTIZATION_BATCH_SIZE, field)

In [7]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        loss = criterion.forward(preds, train_label_batch)
        tot_loss += loss
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)

        if (train_idx + 1) % PRINT == 0:
            tot_acc = 0
            tot_sample = 0
            for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
                # train accuracy
                preds = model.forward(test_data_batch)
                preds = from_galois_to_real_domain(preds, QUANTIZATION_WEIGHT, PRIME)
                pred_args = np.argmax(preds, axis=1)

                tot_acc += np.count_nonzero(pred_args == test_label_batch)
                tot_sample += test_data_batch.shape[0]
            accuracy = tot_acc / tot_sample
            if train_idx != 0:
                tot_loss = tot_loss / PRINT
            print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
            tot_loss = 0

epoch: 1, idx: 1, accuracy: 0.0, loss: 1.1489303588867188
epoch: 2, idx: 1, accuracy: 0.0, loss: 1.1250595092773437
epoch: 3, idx: 1, accuracy: 0.0, loss: 1.1168151855468749
epoch: 4, idx: 1, accuracy: 0.0, loss: 1.0937347412109375
epoch: 5, idx: 1, accuracy: 0.0, loss: 1.0674896240234373
epoch: 6, idx: 1, accuracy: 0.0, loss: 1.039500427246094
epoch: 7, idx: 1, accuracy: 0.3, loss: 1.0040740966796875
epoch: 8, idx: 1, accuracy: 0.2, loss: 0.9877960205078123
epoch: 9, idx: 1, accuracy: 0.1, loss: 0.9959915161132813
epoch: 10, idx: 1, accuracy: 0.2, loss: 0.9797683715820314
epoch: 11, idx: 1, accuracy: 0.3, loss: 0.9596908569335938
epoch: 12, idx: 1, accuracy: 0.3, loss: 0.9365142822265625
epoch: 13, idx: 1, accuracy: 0.5, loss: 0.9221817016601562
epoch: 14, idx: 1, accuracy: 0.6, loss: 0.8920333862304686
epoch: 15, idx: 1, accuracy: 0.6, loss: 0.8736770629882811
epoch: 16, idx: 1, accuracy: 0.6, loss: 0.8635589599609375
epoch: 17, idx: 1, accuracy: 0.6, loss: 0.8335647583007812
epoch: 

KeyboardInterrupt: 